In [1]:
from agent import Agent
from agent import sample_tracjectory, sample_trajectories, collect_trajectories
import numpy as np
import gym
import torch

In [2]:
policy_dict1 = {'ob_dim': 5, 'ac_dim': 2, 'discrete': True, 'size': 3, 'n_layers':5, 'learning_rate': 0.001}
sample_traj1 = {'animate': False, 'max_path_length': 500, 'min_timesteps_per_batch': 10}
return_args1 = {'gamma': 0.9, 'reward_to_go':True, 'nn_baseline':False, 'normalize_advantages': False}
policy_dict2 = {'ob_dim': 5, 'ac_dim': 2, 'discrete': False, 'size': 3, 'n_layers':5, 'learning_rate': 0.001}
return_args2 = {'gamma': 0.9, 'reward_to_go':False, 'nn_baseline':False, 'normalize_advantages': True}

In [3]:
#Build two agents for both test cases.
a1 = Agent(policy_dict1, sample_traj1, return_args1)
a2 = Agent(policy_dict2, sample_traj1, return_args2)

In [4]:
#Build their policy.
a1.build_policy(False)
a2.build_policy(False)

In [5]:
a1.policy(torch.Tensor([[1, 2, 3, 4, 5]]))

tensor([[0.1267, 0.7362]], grad_fn=<AddmmBackward>)

In [6]:
testTensor = torch.tensor([[1.0, 2.0, 0.0,4.,5.], [4., 5., 6.,1.,2.]])
aa1 = a1.sample_action(a1.policy(testTensor))
aa1

tensor([0, 1])

In [7]:
aa2 = a2.sample_action(a2.policy(testTensor))
aa2

tensor([[-0.8048, -0.2212],
        [ 1.4363, -1.4340]])

In [8]:
a1.get_log_prob(a1.policy(testTensor), aa1)

tensor([-1.0592, -0.4240], grad_fn=<SqueezeBackward1>)

In [9]:
a2.get_log_prob(a2.policy(testTensor), aa2)

tensor([[-1.5276, -0.5711],
        [-1.5853, -1.6476]], grad_fn=<SubBackward0>)

In [10]:
lossa1 = a1.calculate_loss(a1.policy(testTensor),aa1, torch.rand(2))
lossa1

tensor(-1.3062, grad_fn=<SumBackward0>)

In [11]:
lossa2 = a2.calculate_loss(a2.policy(testTensor),aa2, torch.rand(2))
lossa2

tensor(-3.7015, grad_fn=<SumBackward0>)

In [12]:
a1.sum_of_rewards([torch.rand(10), torch.rand(5)])

tensor([2.2543, 2.2698, 2.4934, 2.2208, 1.9453, 1.8362, 1.7170, 1.5633, 1.5025,
        0.6620, 2.3047, 1.6790, 1.7620, 0.9241, 0.1591])

In [13]:
a2.sum_of_rewards([torch.rand(10), torch.rand(5)])

tensor([3.7874, 3.7874, 3.7874, 3.7874, 3.7874, 3.7874, 3.7874, 3.7874, 3.7874,
        3.7874, 2.9524, 2.9524, 2.9524, 2.9524, 2.9524])

In [14]:
a2.estimate_return(torch.rand(20,5),[torch.rand(10), torch.rand(10)])

(tensor([3.2745, 3.2745, 3.2745, 3.2745, 3.2745, 3.2745, 3.2745, 3.2745, 3.2745,
         3.2745, 3.3727, 3.3727, 3.3727, 3.3727, 3.3727, 3.3727, 3.3727, 3.3727,
         3.3727, 3.3727]),
 tensor([-0.9747, -0.9747, -0.9747, -0.9747, -0.9747, -0.9747, -0.9747, -0.9747,
         -0.9747, -0.9747,  0.9747,  0.9747,  0.9747,  0.9747,  0.9747,  0.9747,
          0.9747,  0.9747,  0.9747,  0.9747]))

In [15]:
a1.estimate_return(torch.rand(10,5),[torch.rand(10)])

(tensor([2.1984, 2.1961, 2.0828, 1.6100, 1.6407, 1.2491, 1.3053, 1.2550, 0.6516,
         0.6362]),
 tensor([2.1984, 2.1961, 2.0828, 1.6100, 1.6407, 1.2491, 1.3053, 1.2550, 0.6516,
         0.6362]))

In [16]:
#Test trajectory helper functions.
env = gym.make('Pendulum-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [17]:
policy_dict3 = {'ob_dim': 3, 'ac_dim': 1, 'discrete': False, 'size': 3, 'n_layers':5, 'learning_rate': 0.001}
sample_traj3 = {'animate': False, 'max_path_length': 500, 'min_timesteps_per_batch': 10}
return_args3 = {'gamma': 0.9, 'reward_to_go':True, 'nn_baseline':True, 'normalize_advantages': True}

In [18]:
a3 = Agent(policy_dict3, sample_traj3, return_args3)
a3.build_policy(double=True)

In [19]:
sample_tracjectory(a3, env, False)

{'observation': tensor([[-0.9252, -0.3794,  2.0289]], dtype=torch.float64),
 'reward': tensor([-8.5259], dtype=torch.float64),
 'action': tensor([2.4556], dtype=torch.float64)}

In [20]:
paths, num_steps = sample_trajectories(a3,1,env)

In [21]:
obs_no, ac_na, re_n = collect_trajectories(paths)

In [22]:
obs_no

tensor([[ 0.7813, -0.6242,  2.7669],
        [-0.9941, -0.1087, -3.2835],
        [-0.5863,  0.8101,  6.0939],
        [-0.9985, -0.0544, -3.1749],
        [-0.5221,  0.8529, -0.5398],
        [ 0.9255, -0.3787,  1.9358],
        [ 0.0324,  0.9995,  1.3008],
        [-0.7631,  0.6463,  4.1819],
        [ 0.1379, -0.9904,  5.4660],
        [-0.9976, -0.0688, -6.3469],
        [-0.6482, -0.7615,  5.1368]], dtype=torch.float64)

In [23]:
ac_na

tensor([ 1.2667,  3.7430, -2.4929,  1.2624, -2.3471, -1.6284, -0.7426, -0.3957,
         0.9992, -1.0601, -2.1986], dtype=torch.float64)

In [24]:
re_n

[tensor([-1.6360], dtype=torch.float64),
 tensor([-9.3757], dtype=torch.float64),
 tensor([-6.8147], dtype=torch.float64),
 tensor([-9.6051], dtype=torch.float64),
 tensor([-4.6893], dtype=torch.float64),
 tensor([-0.8781], dtype=torch.float64),
 tensor([-2.2157], dtype=torch.float64),
 tensor([-6.3033], dtype=torch.float64),
 tensor([-6.5819], dtype=torch.float64),
 tensor([-11.0809], dtype=torch.float64),
 tensor([-9.8600], dtype=torch.float64)]

In [25]:
r = a3.sum_of_rewards(re_n)

In [26]:
q_n, adv_n = a3.estimate_return(obs_no, re_n)

In [27]:
q_n

tensor([ -1.6360,  -9.3757,  -6.8147,  -9.6051,  -4.6893,  -0.8781,  -2.2157,
         -6.3033,  -6.5819, -11.0809,  -9.8600], dtype=torch.float64)

In [28]:
adv_n

tensor([ 0.8038, -0.7628,  0.0495, -0.7535,  0.9252,  1.0442,  1.1813,  0.3813,
        -0.2567, -2.0926, -0.5196], dtype=torch.float64)

In [29]:
a3.update_parameters(obs_no, ac_na, q_n, adv_n)

In [30]:
env.action_space

Box(1,)

In [31]:
env.observation_space

Box(3,)

In [39]:
ob = env.reset()

In [43]:
a3.policy(torch.tensor(ob)).detach()

tensor([0.0831, 0.4431], dtype=torch.float64)

In [44]:
ob

array([-0.44976461,  0.89314713, -0.92367588])

In [45]:
a3.sample_action(a3.policy(torch.tensor(ob)))

tensor([0.8369], dtype=torch.float64)

In [48]:
ob, rew, done, _ = env.step(a3.sample_action(a3.policy(torch.tensor(ob))).numpy())

In [49]:
done

False